In [11]:
# Install the following version of sqlalchemy to ensure the code in this notebook will
# run as expected.
!pip install sqlalchemy==2.0.22

In [12]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base 
import pickle

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))  # Limit name to 100 characters
    email = Column(String(254), unique=True)  # Limit email to 254 characters
    orders = relationship("Order", back_populates="user")

class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    price = Column(Integer)
    order_items = relationship("OrderItem", back_populates="product")

class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="orders")
    order_items = relationship("OrderItem", back_populates="order")

class OrderItem(Base):
    __tablename__ = 'order_items'
    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey('orders.id'))
    product_id = Column(Integer, ForeignKey('products.id'))
    quantity = Column(Integer)
    order = relationship("Order", back_populates="order_items")
    product = relationship("Product", back_populates="order_items")

## You may replace echo = False to not display debugging messages.
engine = create_engine('sqlite:///ecommerce.db', echo=False)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [2]:
def add_sample_data():
    # Adding users
    user_data = [
        ("Alice", "alice@example.com"),
        ("Bob", "bob@example.com"),
        ("Charlie", "charlie@example.com"),
        ("David", "david@example.com"),
        ("Eve", "eve@example.com"),
        ("Frank", "frank@example.com"),
        ("Grace", "grace@example.com"),
        ("Heidi", "heidi@example.com"),
        ("Ivan", "ivan@example.com"),
        ("Judy", "judy@example.com")
    ]
    for name, email in user_data:
        add_user(name, email)

    # Adding products
    product_data = [
        ("Laptop", 1000),
        ("Smartphone", 500),
        ("Tablet", 300),
        ("Monitor", 150),
        ("Keyboard", 50),
        ("Mouse", 25),
        ("Printer", 200),
        ("Router", 75),
        ("Webcam", 60),
        ("Headphones", 80)
    ]
    for name, price in product_data:
        add_product(name, price)

    # Adding orders
    for i in range(1, 11):
        add_order(i)

    # Adding order_items
    order_item_data = [
        (1, 1, 2),
        (1, 2, 1),
        (2, 3, 1),
        (2, 4, 2),
        (3, 5, 3),
        (3, 6, 1),
        (4, 7, 2),
        (4, 8, 1),
        (5, 9, 1),
        (5, 10, 2),
        (6, 1, 1),
        (6, 2, 2),
        (7, 3, 1),
        (7, 4, 1),
        (8, 5, 2),
        (8, 6, 1),
        (9, 7, 3),
        (9, 8, 1),
        (10, 9, 1),
        (10, 10, 1)
    ]
    for order_id, product_id, quantity in order_item_data:
        add_order_item(order_id, product_id, quantity)

    #Pickle this data
    # Combine data into a single dictionary
    combined_data = {
        'user_data': user_data,
        'product_data': product_data,
        'order_item_data': order_item_data
    }

    # Pickling the combined data to a single file
    with open('sample_data.pkl', 'wb') as f:
        pickle.dump(combined_data, f)

# Call the function to add sample data
add_sample_data()

In [3]:
def print_all_users():
    users = session.query(User).all()
    for user in users:
        print(f"User ID: {user.id}, Name: {user.name}, Email: {user.email}")

# Call the function to print all users
print_all_users()

User ID: 1, Name: Alice, Email: alice@example.com
User ID: 2, Name: Bob, Email: bob@example.com
User ID: 3, Name: Charlie, Email: charlie@example.com
User ID: 4, Name: David, Email: david@example.com
User ID: 5, Name: Eve, Email: eve@example.com
User ID: 6, Name: Frank, Email: frank@example.com
User ID: 7, Name: Grace, Email: grace@example.com
User ID: 8, Name: Heidi, Email: heidi@example.com
User ID: 9, Name: Ivan, Email: ivan@example.com
User ID: 10, Name: Judy, Email: judy@example.com


In [4]:
def print_all_orders():
    orders = session.query(Order).all()
    for order in orders:
        print(f"Order ID: {order.id}")
        print(f"User: {order.user.name} (ID: {order.user_id})")
        print("Order Items:")
        for item in order.order_items:
            product = item.product
            print(f"  - {product.name}: {item.quantity} x ${product.price} = ${item.quantity * product.price}")
        total = sum(item.quantity * item.product.price for item in order.order_items)
        print(f"Total: ${total}")
        print("--------------------")

# Call the function to print all orders
print_all_orders()

Order ID: 1
User: Alice (ID: 1)
Order Items:
  - Laptop: 2 x $1000 = $2000
  - Smartphone: 1 x $500 = $500
Total: $2500
--------------------
Order ID: 2
User: Bob (ID: 2)
Order Items:
  - Tablet: 1 x $300 = $300
  - Monitor: 2 x $150 = $300
Total: $600
--------------------
Order ID: 3
User: Charlie (ID: 3)
Order Items:
  - Keyboard: 3 x $50 = $150
  - Mouse: 1 x $25 = $25
Total: $175
--------------------
Order ID: 4
User: David (ID: 4)
Order Items:
  - Printer: 2 x $200 = $400
  - Router: 1 x $75 = $75
Total: $475
--------------------
Order ID: 5
User: Eve (ID: 5)
Order Items:
  - Webcam: 1 x $60 = $60
  - Headphones: 2 x $80 = $160
Total: $220
--------------------
Order ID: 6
User: Frank (ID: 6)
Order Items:
  - Laptop: 1 x $1000 = $1000
  - Smartphone: 2 x $500 = $1000
Total: $2000
--------------------
Order ID: 7
User: Grace (ID: 7)
Order Items:
  - Tablet: 1 x $300 = $300
  - Monitor: 1 x $150 = $150
Total: $450
--------------------
Order ID: 8
User: Heidi (ID: 8)
Order Items:
  - 

In [5]:
def print_all_products():
    products = session.query(Product).all()
    print("All Products:")
    print("-------------")
    for product in products:
        print(f"ID: {product.id}")
        print(f"Name: {product.name}")
        print(f"Price: ${product.price}")
        print("-------------")

# Call the function to print all products
print_all_products()

All Products:
-------------
ID: 1
Name: Laptop
Price: $1000
-------------
ID: 2
Name: Smartphone
Price: $500
-------------
ID: 3
Name: Tablet
Price: $300
-------------
ID: 4
Name: Monitor
Price: $150
-------------
ID: 5
Name: Keyboard
Price: $50
-------------
ID: 6
Name: Mouse
Price: $25
-------------
ID: 7
Name: Printer
Price: $200
-------------
ID: 8
Name: Router
Price: $75
-------------
ID: 9
Name: Webcam
Price: $60
-------------
ID: 10
Name: Headphones
Price: $80
-------------


In [6]:
def get_orders_by_user(user_id):
    orders = session.query(Order).filter(Order.user_id == user_id).all()
    return orders

# Example usage: get all orders for user with ID 1
user_orders = get_orders_by_user(1)
for order in user_orders:
    print(f"Order ID: {order.id}, User ID: {order.user_id}")

Order ID: 1, User ID: 1


In [7]:
from sqlalchemy import func

# Query to get total quantity sold for each product
product_sales = (
    session.query(
        Product.name,
        func.sum(OrderItem.quantity).label('total_quantity')
    )
    .join(OrderItem, Product.id == OrderItem.product_id)
    .group_by(Product.id)
    .all()
)

for name, total_quantity in product_sales:
    print(f"Product: {name}, Total Quantity Sold: {total_quantity}")

Product: Laptop, Total Quantity Sold: 3
Product: Smartphone, Total Quantity Sold: 3
Product: Tablet, Total Quantity Sold: 2
Product: Monitor, Total Quantity Sold: 3
Product: Keyboard, Total Quantity Sold: 5
Product: Mouse, Total Quantity Sold: 2
Product: Printer, Total Quantity Sold: 5
Product: Router, Total Quantity Sold: 2
Product: Webcam, Total Quantity Sold: 2
Product: Headphones, Total Quantity Sold: 3


In [9]:
!pip install dogpile.cache


   ---------------------------------------- 0/3 [pbr]
   ------------- -------------------------- 1/3 [stevedore]
   -------------------------- ------------- 2/3 [dogpile.cache]
   ---------------------------------------- 3/3 [dogpile.cache]



In [11]:
import time
from dogpile.cache import make_region
from sqlalchemy.orm import Session

# Configure the cache region
region = make_region().configure(
    'dogpile.cache.memory',
    expiration_time=3600
)

@region.cache_on_arguments()
def get_all_products_cached():
    with Session(engine) as session:
        query = session.query(Product)
        return [
            {
                'id': product.id,
                'name': product.name,
                'price': product.price
            } for product in query.all()
        ]

def get_all_products_uncached():
    with Session(engine) as session:
        query = session.query(Product)
        return [
            {
                'id': product.id,
                'name': product.name,
                'price': product.price
            } for product in query.all()
        ]

def compare_execution_times(num_iterations=5):
    print(f"Comparing execution times over {num_iterations} iterations:")
    
    # Warm up the cache
    get_all_products_cached()
    
    # Test cached function
    cache_start_time = time.time()
    for _ in range(num_iterations):
        get_all_products_cached()
    cache_end_time = time.time()
    cache_total_time = cache_end_time - cache_start_time
    
    # Test uncached function
    uncache_start_time = time.time()
    for _ in range(num_iterations):
        get_all_products_uncached()
    uncache_end_time = time.time()
    uncache_total_time = uncache_end_time - uncache_start_time
    
    print(f"Cached function total time: {cache_total_time:.6f} seconds")
    print(f"Cached function average time: {cache_total_time/num_iterations:.6f} seconds")
    print(f"Uncached function total time: {uncache_total_time:.6f} seconds")
    print(f"Uncached function average time: {uncache_total_time/num_iterations:.6f} seconds")
    if cache_total_time > 0:
        print(f"Speed improvement: {uncache_total_time/cache_total_time:.2f}x faster with caching")
    else:
        print("Speed improvement: Cache execution time is too fast to measure (zero).")

# Run the comparison
compare_execution_times()


Comparing execution times over 5 iterations:
Cached function total time: 0.000000 seconds
Cached function average time: 0.000000 seconds
Uncached function total time: 0.002458 seconds
Uncached function average time: 0.000492 seconds
Speed improvement: Cache execution time is too fast to measure (zero).


In [15]:
from sqlalchemy import text


with Session() as s:
    stmt = s.query(Product).statement
    explain_stmt = f"EXPLAIN QUERY PLAN {str(stmt)}"
    result = s.execute(text(explain_stmt))
    print("EXPLAIN QUERY PLAN for get_all_products_cached():")
    for row in result:
        print(row)

EXPLAIN QUERY PLAN for get_all_products_cached():
(2, 0, 0, 'SCAN products')
